<a href="https://colab.research.google.com/github/ryannhally/Comps_Project_and_Essay/blob/main/CS_390_Pretrain_in_ES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets evaluate sacrebleu elotl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behav

In [ ]:
from datasets import load_dataset
from datasets import Dataset

# Spanish - English

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
idx = []
translation = []

i = 0
with open('/content/drive/MyDrive/spa.txt', 'r') as anki_txt:
  for line in anki_txt:
    line = line.split('\t')
    idx.append(i)
    translation.append({'es': line[1], 'en': line[0]})
    i += 1


In [ ]:
# Check
print(translation[0])
print(translation[100000])

{'es': 'Ve.', 'en': 'Go.'}
{'es': 'Cuando era chica, me encantaban los chupetines.', 'en': 'I loved lollipops when I was little.'}


In [ ]:
# Using only 20,000 examples
translation = translation[:100000]
idx = idx[:100000]

In [ ]:
# Check
print(len(translation))

100000


In [ ]:
# Make dictionary
data = { "id": idx, "translation": translation}

books = Dataset.from_dict(data)
books = books.train_test_split(test_size=0.1)

In [ ]:
print(books)

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 90000
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 10000
    })
})


In [ ]:
# Check
print(books['train'][0])
print(books['train'][18018] )
print(books['train'][35000])
print(books['train'][500])
print(books['test'][500])

{'id': 38913, 'translation': {'en': "He's rich and powerful.", 'es': 'Es rico y poderoso.'}}
{'id': 25045, 'translation': {'en': "I couldn't kill Tom.", 'es': 'No pude matar a Tom.'}}
{'id': 27534, 'translation': {'en': 'Tom listens to Mary.', 'es': 'Tom escucha a Mary.'}}
{'id': 57251, 'translation': {'en': "Tom hasn't been there yet.", 'es': 'Tom no ha estado todavía allí.'}}
{'id': 71208, 'translation': {'en': 'She laid her baby on the bed.', 'es': 'Ella dejó a su niño en la cama.'}}


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

tokenizer = AutoTokenizer.from_pretrained('google-t5/t5-small')
model = AutoModelForSeq2SeqLM.from_pretrained('google-t5/t5-small')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Sign into HuggingFace
from google.colab import userdata
userdata.get('HuggingFaceKey')

from huggingface_hub import notebook_login

notebook_login('HuggingFaceKey')

In [ ]:
source_lang = 'es'
target_lang = 'en'
prefix = 'translate Spanish to English: '


def preprocess_function(examples):

    inputs = [prefix + example[source_lang] for example in examples["translation"]]

    targets = [example[target_lang] for example in examples["translation"]]

    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    with tokenizer.as_target_tokenizer():

        labels = tokenizer(targets, max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [ ]:
tokenized_books = books.map(preprocess_function, batched=True)

Map:   0%|          | 0/90000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
# Check
print(tokenized_books)

DatasetDict({
    train: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 90000
    })
    test: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
})


In [ ]:
# Check
print(tokenized_books['train'][0])
print(tokenized_books['train'][1])
print(tokenized_books['train'][200])
print(tokenized_books['train'][30000])

{'id': 38913, 'translation': {'en': "He's rich and powerful.", 'es': 'Es rico y poderoso.'}, 'input_ids': [13959, 5093, 12, 1566, 10, 1122, 3, 2234, 32, 3, 63, 9489, 15, 1859, 32, 5, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [216, 31, 7, 2354, 11, 2021, 5, 1]}
{'id': 98468, 'translation': {'en': "Tom was surprised by Mary's candor.", 'es': 'Tom se sorprendió por el candor de Mary.'}, 'input_ids': [13959, 5093, 12, 1566, 10, 3059, 142, 78, 52, 2026, 727, 23, 4922, 5569, 3, 15, 40, 2988, 127, 20, 3790, 5, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [3059, 47, 5597, 57, 3790, 31, 7, 2988, 127, 5, 1]}
{'id': 6888, 'translation': {'en': 'Who sent this?', 'es': '¿Quién mandó esto?'}, 'input_ids': [13959, 5093, 12, 1566, 10, 3, 2, 5991, 23, 154, 29, 388, 26, 4922, 259, 32, 58, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [2645, 1622, 48, 58, 1]}
{'id

In [ ]:
# Data collator
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
# Hyperparemeters
training_args = Seq2SeqTrainingArguments(

    output_dir="ryannhally/t5-small-nah-es",

    evaluation_strategy="steps",

    eval_steps=500,

    save_steps=500,

    learning_rate=2e-5,

    per_device_train_batch_size=16,

    per_device_eval_batch_size=16,

    weight_decay=0.01,

    save_total_limit=3,

    num_train_epochs=10,

    fp16=True,

    push_to_hub=True

)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Seq2SeqTrainer(

    model=model,

    args=training_args,

    train_dataset=tokenized_books["train"],

    eval_dataset=tokenized_books["test"],

    tokenizer=tokenizer,

    data_collator=data_collator,

)

<ipython-input-46-7c93e7e4f211>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
500,2.317700,2.040578
1000,2.244400,1.931430
1500,2.141900,1.850496
2000,2.078500,1.787002
2500,2.026800,1.726825
3000,1.972300,1.673268
3500,1.934700,1.627573
4000,1.874900,1.588976
4500,1.849200,1.550001
5000,1.805800,1.516334


TrainOutput(global_step=56250, training_loss=1.328538952907986, metrics={'train_runtime': 7338.1051, 'train_samples_per_second': 122.647, 'train_steps_per_second': 7.665, 'total_flos': 6309084958556160.0, 'train_loss': 1.328538952907986, 'epoch': 10.0})

# Spanish - Nahuatl

In [ ]:
# Load model and tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

tokenizer = AutoTokenizer.from_pretrained('ryannhally/t5-small-nah-es')
model = AutoModelForSeq2SeqLM.from_pretrained('ryannhally/t5-small-nah-es')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [ ]:
# Load axolotl dataset
axolotl_raw = load_dataset('somosnlp-hackathon-2022/Axolotl-Spanish-Nahuatl')
axolotl_filter = axolotl_raw.filter(lambda x:x['sp'] is not None)
axolotl = axolotl_filter.filter(lambda x:x['src'] != 'opus-bible-uedin')

README.md:   0%|          | 0.00/2.77k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/6.45M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20028 [00:00<?, ? examples/s]

Filter:   0%|          | 0/20028 [00:00<?, ? examples/s]

Filter:   0%|          | 0/20024 [00:00<?, ? examples/s]

In [ ]:
# Check
print(axolotl.num_rows)

{'train': 12203}


In [ ]:
# Normalize
import elotl.corpus
import elotl.nahuatl.orthography
n = elotl.nahuatl.orthography.Normalizer("sep")

translation = []
idx = []

i = 0
for example in axolotl['train']:
  idx.append(i)
  translation.append({'es': axolotl['train'][i]['sp'], 'na': n.normalize(axolotl['train'][i]['nah'])})
  i += 1

In [ ]:
# Load bible dataset
bible_raw = load_dataset('somosnlp-hackathon-2022/Axolotl-Spanish-Nahuatl')
bible_filter = bible_raw.filter(lambda x:x['sp'] is not None)
bible = bible_filter.filter(lambda x:x['src'] == 'opus-bible-uedin')

Filter:   0%|          | 0/20024 [00:00<?, ? examples/s]

In [ ]:
# Normalize bible dataset
index = 0
for example in bible['train']:
  idx.append(i)
  translation.append({'es': bible['train'][index]['sp'], 'na': n.normalize(bible['train'][index]['nah'])})
  i +=1
  index +=1

In [ ]:
# Check
print(len(idx))
print(len(translation))
print(translation[0])
print(translation[13000])

20024
20024
{'es': 'Y así, cuando hizo su ofrenda de fuego, se sienta delante de los demás y una persona se queda junto a él.', 'na': 'au in ye yujki in on tlenamakak niman ye ik teixpan on motlalia se tlakatl itech mokaua.'}
{'es': ' "¡Ay de vosotros, escribas y fariseos, hipócritas!Porque edificáis los sepulcros de los profetas y adornáis los monumentos de los justos', 'na': "'¡lástima para nemejuamej temachtijkej ikan itlanauatil dios niman nemejuamej fariseos, nenomexayakyejkej!lástima, pampa nenkinchijchiuiliaj inmikakal on tiotlajtojkej, niman nenkitlatlajmachyotiaj ne kampa toktokej on yejuan kuajkualtin tlakamej katkaj."}


In [ ]:
# Make dictionary
data = { "id": idx, "translation": translation}

books = Dataset.from_dict(data)
books = books.train_test_split(test_size=0.1)

In [ ]:
# Check
print(books)
print(books['train'][0])
print(books['train'][18018] )
print(books['train'][500])

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 18021
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 2003
    })
})
{'id': 1399, 'translation': {'es': 'Hay tres clases: la col bolita, la col de hojas anchas y la col pequeña.', 'na': 'onkak eyi taman: kolex bolita, mapapatauakkolex uan talpankolex.'}}
{'id': 14779, 'translation': {'es': ' Cuando Jesús llegó a aquel lugar, alzando la vista le vio y le dijo: --Zaqueo, date prisa, desciende; porque hoy es necesario que me quede en tu casa', 'na': 'ijkuak jesús ompaka panotiaya, onajkotlachix, niman okijlij sakeo ―zaqueo, xuajtemotetsi, pampa aman ika onkaj mochan ninokauas.'}}
{'id': 3006, 'translation': {'es': 'Se hará ofrenda de flores,con ellas se ataviará, el Guerrero del sur.', 'na': 'xochitl mantiu,ika momakixtia in uitsnauatl yaotl.'}}


In [ ]:
# Define preprocessing function
source_lang = "es"

target_lang = "na"

prefix = "translate Spanish to Nahuatl: "


def preprocess_function(examples):

    inputs = [prefix + example[source_lang] for example in examples["translation"]]

    targets = [example[target_lang] for example in examples["translation"]]

    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    with tokenizer.as_target_tokenizer():

        labels = tokenizer(targets, max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [ ]:
# Preprocess
tokenized_books = books.map(preprocess_function, batched=True)

Map:   0%|          | 0/18021 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2003 [00:00<?, ? examples/s]

In [ ]:
# Check
print(tokenized_books)
print(tokenized_books['train'][0])
print(tokenized_books['train'][1])
print(tokenized_books['train'][200])
print(tokenized_books['train'][18000])

DatasetDict({
    train: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 18021
    })
    test: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2003
    })
})
{'id': 1399, 'translation': {'es': 'Hay tres clases: la col bolita, la col de hojas anchas y la col pequeña.', 'na': 'onkak eyi taman: kolex bolita, mapapatauakkolex uan talpankolex.'}, 'input_ids': [13959, 5093, 12, 22885, 76, 144, 40, 10, 8567, 3, 929, 7, 25062, 7, 10, 50, 7632, 12903, 17, 9, 6, 50, 7632, 20, 3534, 1191, 7, 46, 3441, 7, 3, 63, 50, 7632, 158, 835, 2, 9, 5, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [30, 1258, 157, 3, 15, 63, 23, 3, 17, 9, 348, 10, 3, 11292, 994, 12903, 17, 9, 6, 2828, 9, 4665, 402, 1639, 11292, 994, 3, 76, 152, 3, 1947, 2837, 11292, 994, 5, 1]}
{'id': 15721, 'trans

In [ ]:
# Hyperparemeters
training_args = Seq2SeqTrainingArguments(

    output_dir="ryannhally/t5-small-nah-es-2",

    evaluation_strategy="steps",

    eval_steps=1000,

    save_steps=5000,

    learning_rate=2e-5,

    per_device_train_batch_size=16,

    per_device_eval_batch_size=16,

    weight_decay=0.01,

    num_train_epochs=280,

    save_total_limit=3,

    fp16=True,

    push_to_hub=True



)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Seq2SeqTrainer(

    model=model,

    args=training_args,

    train_dataset=tokenized_books['train'],

    eval_dataset=tokenized_books['test'],

    tokenizer=tokenizer,

    data_collator=data_collator,


)

<ipython-input-18-3245eda40cf6>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
1000,3.495500,3.107437
2000,3.148500,2.829293
3000,2.967500,2.678507
4000,2.846200,2.568168
5000,2.739200,2.479841
6000,2.676100,2.412693
7000,2.598700,2.354055
8000,2.546700,2.306469
9000,2.509600,2.265082
10000,2.450800,2.224265


TrainOutput(global_step=315560, training_loss=1.5698560808046127, metrics={'train_runtime': 46329.1301, 'train_samples_per_second': 108.914, 'train_steps_per_second': 6.811, 'total_flos': 1.661980872231813e+17, 'train_loss': 1.5698560808046127, 'epoch': 280.0})